In [1]:
import pandas as pd
import re
# coding: utf-8

# 만든 기능 임포트
from crawling import UrlCrawling, ContentCrawling
from preprocessing import Preprocessing
from clustering import Clustering
from summary import Summary
from remove import Remove
from database import insert, selectAll, selectToDay
%matplotlib agg
from draw import drawKeyword
from tts import tts

In [ ]:
# url 크롤링
url_crawler = UrlCrawling()     # 기사 링크 크롤링 객체

six_url_df = url_crawler.getSixUrl()                            # 6개 카테고리 (카테고리, url 데이터프레임)
entertainment_url_df = url_crawler.getEntertainmentUrl()        # 연예 (카테고리, url 데이터프레임)
sports_url_df = url_crawler.getSportsUrl()                      # 스포츠 (카테고리, url 데이터프레임)

Remove.duplication(six_url_df, entertainment_url_df, sports_url_df)     # 이미 요약한 기사 제거

# 본문 크롤링
six_url, entertainment_url, sports_url = list(six_url_df['six_url']), list(entertainment_url_df['entertainment_url']), list(sports_url_df['sports_url'])
all_url = six_url + entertainment_url + sports_url      # 전체 url
category = list(six_url_df['category']) + list(entertainment_url_df['category']) + list(sports_url_df['category'])

content_crawler = ContentCrawling()     # 기사 크롤링 객체

content_crawler.getSixContent(six_url)                                          # 6개 카테고리 기사 크롤링
content_crawler.getEntertainmentContent(entertainment_url)                      # 연예 기사 크롤링
content_crawler.getSportsContent(sports_url)                                    # 스포츠 기사 크롤링

news_df = content_crawler.makeDataFrame(all_url, category)                      # 본문 데이터프레임 생성

Remove.photoNews(news_df)                                                       # 포토 기사 삭제
Remove.shortNews(news_df)                                                       # 3문장 or 300자 이하 기사 삭제

Preprocessing.getNouns(news_df)                                                 # 명사 추출

Remove.englishNews(news_df)                                                     # 영어 기사 삭제

vector_list = Preprocessing.getVector(news_df)                                  # 명사 벡터화

Clustering.addClusterNumber(news_df, vector_list)                               # 군집 번호 열 생성
cluster_counts_df = Clustering.getClusteredArticle(news_df)                     # 군집 개수 카운트한 df

summary_news = Summary.getSummaryArticle(news_df, cluster_counts_df)            # 기사 요약

Preprocessing.convertCategory(summary_news)                                     # 카테고리 이름 변환

Remove.similarNews(summary_news)                                                # 오늘 요약한 기사와 유사한 기사 drop

drawKeyword()

In [ ]:
insert(summary_news)

In [27]:
txt = """지난해 명품 구매 건수 기준으로 전체 연령층에서 MZ세대가 차지한 비중이 절반을 넘어선 것이다.

MZ세대는 에르메스·샤넬 등 초고가 명품보다 가격이 낮은 럭셔리 브랜드로 개성을 표현하려는 경향을 보였으며 경기 침체 속에서도 이 같은 분위기가 이어졌다.

5일 매일경제가 확보한 롯데멤버스의 소비자 분석 보고서에 따르면 지난해 명품 소비자 중 Z세대(1995~2003년생)가 17.4%, 밀레니얼 세대(1980~1994년생)가 38.5%를 각각 차지했다.

밀레니얼 세대에서도 월 1회 이상 명품을 구매하는 비율이 22.8%로 나타났다.
"""

def getKeyword(summary_content):

    stop_word = ["지난해", "지난달", "대부분"]
    nouns = []

    for keyword, tag in hannanum.pos(txt):    # 3글자 이상 명사
        if tag in ['N'] and len(keyword) >= 3 and keyword not in stop_word:
            nouns.append(keyword)

    vectorizer = TfidfVectorizer(min_df=1)  # min_df 특정 단어가 나타나는 '문서의 수'
    vectorizer.fit(nouns)

    top_keyword = Counter(vectorizer.vocabulary_).most_common(10)   # 상위 10개

    keyword_list = []
    for keyword in top_keyword:
        keyword_list.append(keyword[0])

    return ",".join(keyword_list)

print(getKeyword(txt))

연령층,에르메스,소비자,샤넬,브랜드,분위기,보고서,밀레니얼,매일경제,롯데멤버스


In [ ]:
# 시각화
from draw import drawKeyword
drawKeyword()